# 📓 The GenAI Revolution Cookbook

**Title:** Understanding the Agent Communication Protocol (ACP): Why Agents Need a Common Language

**Description:** Eliminate brittle integrations. Use the Agent Communication Protocol (ACP) to enable heterogeneous agents to interoperate with performatives, discovery, routing, lifecycle.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Integrating agents from different frameworks like LangChain, CrewAI, and SmolAgents usually means writing N×M adapters. Each new agent adds more one-off glue. Small API changes break workflows, causing a maintenance tax that grows faster than your system. If you're looking for a practical approach to orchestrating multi-agent systems with CrewAI, our guide on [how to build multi-agent AI systems with CrewAI and YAML](/blog/44830763/how-to-build-multi-agent-ai-systems-with-crewai-and-yaml-2) provides step-by-step patterns and reusable templates.

A minimal Agent Communication Protocol (ACP) solves this by standardizing how agents express intent, track conversations, and route messages—without forcing you to rewrite your stack. This explainer focuses on **one core concept**: a minimal message envelope with performatives and conversation threading that reduces coupling and makes multi-agent systems reliable.

## Why This Matters

When a planner agent asks a retriever for documents, the retriever might return results, refuse the request, or report an error. Without a shared vocabulary for these outcomes, the planner must guess intent from HTTP status codes, custom JSON keys, or framework-specific wrappers. Add a third agent and the permutations explode.

**Concrete symptoms:**

- Silent delegation loops when agents misinterpret refusals as acknowledgments
- Retry storms because correlation IDs are missing or inconsistent
- Debugging nightmares when conversation context evaporates across hops

**Cost of ignoring:** Every new agent requires custom adapters for every existing agent. A four-agent system needs twelve integration points; a ten-agent system needs ninety. Maintenance time grows quadratically, and small API changes cascade into breaking changes.

**The problem:** Ad hoc message formats mean no reliable way to distinguish a request from an acknowledgment, no conversation identifiers to tie related messages together, and no routing metadata to select the right agent without hardcoding endpoints.

## How It Works

ACP introduces three mechanisms that work together to align agent communication:

### 1. Performatives as Speech Acts

Performatives describe **intent** in a message. A small, fixed set—request, inform, propose, acknowledge, refuse, error—covers most agent interactions. Each performative signals what the sender expects and what the receiver should do next.

- **request**: sender needs a response (e.g., "retrieve documents matching query X")
- **inform**: sender provides data without expecting action (e.g., "here are three documents")
- **refuse**: receiver declines to fulfill a request (e.g., "query exceeds rate limit")
- **error**: receiver encountered a failure (e.g., "database timeout")

By enforcing allowed transitions (e.g., request → {inform | refuse | error}), you prevent silent misinterpretations. A planner that sends a request knows to expect inform, refuse, or error—not an ambiguous 200 OK with no payload.

### 2. A Minimal JSON Envelope

Every message wraps its payload in a standard envelope with required fields:

```json
{
  "performative": "request",
  "conversation_id": "conv-abc123",
  "correlation_id": "msg-001",
  "sender": "planner-agent",
  "receiver": "retriever-agent",
  "subject": "document.retrieve",
  "parts": [
    {
      "content_type": "application/json",
      "data": {"query": "RAG patterns", "limit": 5}
    }
  ]
}
```

- **conversation_id**: ties all messages in a multi-turn exchange (e.g., request → inform → acknowledge)
- **correlation_id**: unique per message; enables idempotency and tracing
- **subject**: semantic label for routing (e.g., "document.retrieve", "task.plan")
- **parts**: array of content blocks, each with content_type and data; supports multimodal payloads (text, JSON, images)

This envelope separates **protocol metadata** (who, what, why) from **payload data** (the actual query or result), making it easy to validate, route, and trace messages without parsing application-specific schemas.

### 3. Conversation and Correlation IDs for Reliability

**conversation_id** groups related messages into a logical thread. When a planner sends a request and the retriever replies with inform, both share the same conversation_id. Operators can reconstruct the full exchange in logs or traces, even if messages cross multiple agents.

**correlation_id** uniquely identifies each message. Receivers use it to detect duplicates (idempotency) and to link responses back to requests. If a planner retries a request after a timeout, the retriever can recognize the duplicate correlation_id and return the cached response instead of re-executing the query.

Together, these IDs turn opaque message streams into traceable, resumable conversations. For a deeper dive into building agents that maintain state and context across interactions, see our [step-by-step guide to building a stateful AI agent with LangGraph](/blog/44830763/how-to-build-a-stateful-ai-agent-with-langgraph-step-by-step-5).

### 4. Routing by Metadata, Not Hardcoded Endpoints

Instead of hardcoding "send retrieval requests to http://retriever:8080", agents declare **subjects** they handle (e.g., "document.retrieve") and **capabilities** they provide (e.g., "vector_search"). A lightweight registry maps subjects to agent endpoints, and a router selects targets based on metadata in the envelope.

When a planner sends a request with subject "document.retrieve", the router queries the registry, finds the retriever agent, and forwards the message. If you swap the retriever for a new implementation, you update one registry record—not every sender.

```mermaid
sequenceDiagram
    participant Planner
    participant Router
    participant Registry
    participant Retriever

    Planner->>Router: request (subject: document.retrieve, conversation_id: conv-abc123, correlation_id: msg-001)
    Router->>Registry: lookup(subject: document.retrieve)
    Registry-->>Router: endpoint: http://retriever:8080
    Router->>Retriever: forward request
    Retriever-->>Router: inform (conversation_id: conv-abc123, correlation_id: msg-002, in_reply_to: msg-001)
    Router-->>Planner: forward inform
```

Semantic alignment is not optional; it's the foundation of reliable coordination. For a deeper understanding of how protocol standards can drive agent interoperability and safety, see our breakdown of the [Model Context Protocol (MCP)](/blog/44830763/model-context-protocol-mcp-explained-2025-guide-for-builders).

## What You Should Do

### 1. Define and Enforce a Minimal Performative Set

Start with six performatives: request, inform, propose, acknowledge, refuse, error. Define allowed transitions (e.g., request → {inform | refuse | error}) and validate incoming messages at ingress with JSON Schema. Reject messages with unsupported performatives or invalid transitions.

**Pseudo-config:**

In [ ]:
performatives = {request, inform, propose, acknowledge, refuse, error}
transitions = {
  request: [inform, refuse, error],
  propose: [acknowledge, refuse],
  inform: [acknowledge]
}
validate_envelope(message):
  assert message.performative in performatives
  if message.in_reply_to:
    assert message.performative in transitions[parent.performative]

**Measurement:** Track the proportion of unsupported_performative and invalid_transition errors. If >5% of messages fail validation, your performative set is incomplete or agents are sending malformed messages.

### 2. Standardize a JSON Envelope with Required IDs

Wrap every message in an envelope with performative, conversation_id, correlation_id, sender, receiver, subject, and parts. Generate conversation_id once per logical exchange (e.g., when a planner starts a new task) and propagate it across all related messages. Generate a unique correlation_id for each message.

**Pseudo-config:**

In [ ]:
envelope = {
  performative: "request",
  conversation_id: generate_uuid(),  // once per conversation
  correlation_id: generate_uuid(),   // unique per message
  sender: "planner-agent",
  receiver: "retriever-agent",
  subject: "document.retrieve",
  parts: [{content_type: "application/json", data: {...}}]
}

**Measurement:** Log conversation_id and correlation_id in structured logs. Measure retry rates keyed by correlation_id; if >10% of messages are retried, investigate timeout settings or idempotency handling. Track end-to-end latency distributions per performative to identify slow paths.

### 3. Introduce a Lightweight Registry and Route by Subject

Deploy a simple registry (e.g., Redis, Consul, or an in-memory map) that maps subjects to agent endpoints. Agents register on startup with their subjects and capabilities. A router reads the envelope's subject, queries the registry, and forwards the message to the matching agent.

**Pseudo-config:**

In [ ]:
registry.register(agent_id="retriever-agent", subjects=["document.retrieve"], endpoint="http://retriever:8080")
router.route(message):
  targets = registry.lookup(subject=message.subject)
  forward(message, targets[0].endpoint)

**Measurement:** Track registry lookup latency and cache hit rates. If lookup latency >10ms, add caching. Monitor the number of subjects per agent; if >10, consider splitting agents by capability. Set TTLs on registry records and alert if stale entries exceed 5% of total.

## Conclusion – Key Takeaways

A minimal ACP—performatives, a standard envelope, conversation/correlation IDs, and routing by subject—eliminates N×M adapter sprawl and makes multi-agent systems traceable and reliable. You can adopt it incrementally: start with performatives and envelopes in one agent pair, then add the registry as your topology grows.

**When to Care:**

- You have more than two frameworks (LangChain, CrewAI, SmolAgents) in one system
- You have more than three agents and see retry loops or silent failures in logs
- You swap agent implementations weekly and want to avoid rewriting senders
- You need to reconstruct conversation history or debug delegation chains in production

Start with performatives and envelopes today. Add the registry when you have more than five subjects or swap agents frequently. Measure validation error rates, retry rates by correlation_id, and end-to-end latency per performative to confirm the protocol is reducing coupling and improving reliability.